## Settings

### DC power supply

In [10]:
import pyvisa

#Create Resource manager instance for DC power supply
rm= pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL9::INSTR')

In [12]:
# Start the communication by VISA identifying resource name
rnd = rm.open_resource('ASRL9::INSTR')

#Checking the machine name
print(rnd.query('*IDN?'))

RND 320-KD3305P V4.2 SN:09924475



#### Definition of Stabilized Power Supply Operating Functions

In [13]:
#通信インスタンスとチャンネルと電圧と電流を指定して出力オン
def out_put_on(sps,ch,vol,cur):
    sps.write("VSET"+str(ch)+":"+str(vol)) #set ch1 output current 0.01V
    sps.write("ISET"+str(ch)+":"+str(cur)) #set ch1 output current 0.01A
    sps.write("OUT1")

def out_put_on_only_V(sps,ch,vol):
    sps.write("VSET"+str(ch)+":"+str(vol)) #set ch1 output current 0.01V
    sps.write("ISET"+str(ch)+":0") #set ch1 output current 0A
    sps.write("OUT1")

#通信インスタンスとチャンネル番号を指定して出力オフ
def out_put_off(sps,ch):
    sps.write("VSET"+str(ch)+":0") 
    sps.write("ISET"+str(ch)+":0") 
    sps.write("OUT1")

def out_put_off_all(sps):
    sps.write("VSET1:0") 
    sps.write("ISET1:0") 
    sps.write("VSET2:0") 
    sps.write("ISET2:0") 
    sps.write("OUT1")

### Magnet meter

In [14]:
import serial
import serial.tools.list_ports
import time
import pandas as pd

# Find and configure the serial port
def configure_serial():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        if "Prolific PL2303GC USB Serial COM Port" in port.description:
            return serial.Serial(
                port.device,
                baudrate=9600,
                parity=serial.PARITY_NONE,
                stopbits=serial.STOPBITS_ONE,
                bytesize=serial.EIGHTBITS,
                timeout=1
            )
    raise IOError("Prolific PL2303GC USB Serial COM Port not found")

ser = configure_serial()

# コマンドを送信して応答が#ok!であるか確認する関数
def send_command_with_ok_check(command, retries=5):
    for attempt in range(retries):
        ser.write((command + '\r\n').encode())
        print(f"Sent command: {command}")  # デバッグ用
        time.sleep(1)
        response = ser.read_all()
        print(f"Response: {response.decode('latin-1')}")  # デバッグ用
        if b'#ok!' in response:
            return True
        print(f"Retry {attempt + 1}/{retries} for command: {command}")
    return False

def send_receive_command(command):
    try:
        ser.write((command + '\r\n').encode())
        print(f"Sent command: {command}")  # デバッグ用
        time.sleep(1)  # 応答の待機時間を増やす
        response = ser.read_all()  # 全ての応答を読み取る
        return response
    except serial.SerialException as e:
        print(f"Serial exception: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

# データクリアを送信
if not send_command_with_ok_check("$C!"):
    print("Failed to clear data")

# 記録間隔を設定する
if not send_command_with_ok_check("$Y000001!"):
    print("Failed to set record interval")


Sent command: $C!
Response: #ok!#UFFFF18051E0C0D2200E2010000!#S000100E2011500420011!#S000000E2011500420011!
Sent command: $Y000001!
Response: #ok!#UFFFF18051E0C0D2300E2010001!#UFFFF18051E0C0D2400E2010001!


### Control board of Ultimaker (Teensy4.0)

In [15]:
import serial.tools.list_ports

# COMポートの設定（Teensy4.0のものを自動選択）

# 利用可能なポートリストを取得
ports = serial.tools.list_ports.comports()
portList = []

for one in ports:
    portList.append(str(one))
    print(str(one))

# "Serielles USB-Gerät" を含むポートを自動選択
use = None
for port in portList:
    if "Serielles USB-Gerät" in port:
        print(f"Selected Port: {port}")
        use = port.split(' ')[0]  # ポート名を抽出（例：COM3）
        break

if use is None:
    print("No suitable port found.")
else:
    print(f"Using port: {use}")

# COMポートが見つかった場合の後続処理をここに追加

# Portrate setting
baudrate = 115200

if use is None:
    print("Selected COM port not found.")
else:
    # Make serial port instance
    try:
        serialInst = serial.Serial(use, baudrate, timeout=1) 
        print("Serial port opened successfully.")
    except serial.SerialException as e:
        print(f"Error opening serial port: {e}")
        exit()

COM6 - Prolific PL2303GC USB Serial COM Port (COM6)
COM3 - Intel(R) Active Management Technology - SOL (COM3)
COM5 - Serielles USB-Gerät (COM5)
COM9 - Serielles USB-Gerät (COM9)
Selected Port: COM5 - Serielles USB-Gerät (COM5)
Using port: COM5
Serial port opened successfully.


## Adjustment of position

### Home (Please not use: Limit Switch is not working now)

In [16]:
#serialInst.write(('H').encode('utf-8'))

### X-axis (Enter arbitrary distance [mm])

In [18]:
displacement = str(int(input("X axis Distance (mm)")) * 2 / 3) #adjust the number according to the result
serialInst.write(('X' + displacement).encode('utf-8'))

19

### Y-axis (Enter arbitrary distance [mm])

In [ ]:
displacement = str(int(input("Y axis Distance (mm)")) * 2 / 3) #adjust the number according to the result
serialInst.write(('Y' + displacement).encode('utf-8'))

### Z-axis (Enter arbitrary distance [mm])

In [ ]:
displacement = str(int(input("X axis Distance (mm)")) * 2 / 3) #adjust the number according to the result
serialInst.write(('Z' + displacement).encode('utf-8'))

## Measurement

### DC power supply

In [ ]:
#DCDC converter input active voltage
out_put_on(rnd,1,5,0.1) # channel 1, 5 V, 0.1 A

### Stepping motor move

In [3]:
import csv
import os
import time
import serial.tools.list_ports
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# Save folder
folder_path = "C:\\Users\\kary\\Documents\\9_data\\Electromagnet\\" # change path for yourself
file_path = os.path.join(folder_path, "position_data.csv")

# Initialize CSV file
def initialize_csv():
    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "High_Precision_Timestamp", "X_Position", "Y_Position"])

# 位置情報のCSVファイルへの保存
def save_position_to_csv(x, y):
    now = datetime.now().strftime("%m/%d/%Y %H:%M:%S.%f")[:-3]
    high_precision_time = time.perf_counter()
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([now, high_precision_time, x, y])

# メッシュ設定
x_mesh_size = 1.0  # mm
y_mesh_size = 1.0  # mm
resolution = 0.5  # mm
x_num_points = int(x_mesh_size / resolution + 1)
y_num_points = int(y_mesh_size / resolution + 1)

def move_motor(axis, displacement):
    command = axis + str(displacement)
    serialInst.write(command.encode('utf-8'))
    # モーターが動作するための待機時間（調整が必要）
    time.sleep(1)

def run_mesh_movement():
    current_x = 0.0
    current_y = 0.0
    initialize_csv()
    save_position_to_csv(current_x, current_y)  # 初期位置 (0, 0) を記録
        
    for y in range(y_num_points):
        for x in range(x_num_points):
            target_y = y * resolution
            target_x = x * resolution

            # Y軸の相対位置を計算
            if target_y != current_y:
                move_motor('Y', target_y - current_y)
                current_y = target_y
            
            # X軸の相対位置を計算
            if target_x != current_x:
                move_motor('X', target_x - current_x)
                current_x = target_x

            # 物体が位置に到達するのを待つ（調整が必要）
            time.sleep(1)
            
            # シリアルポートからデータを読み取る
            try:
                if serialInst.in_waiting > 0:
                    data = serialInst.readline().decode('utf-8').strip()
                    if data == "Stopped":
                        save_position_to_csv(current_x, current_y)
                        print(f"Received data: {data} at X: {current_x}, Y: {current_y}")
                    else:
                        print(f"Unexpected data: {data}")
                else:
                    print("No data received.")
            except serial.SerialException as e:
                print(f"Error reading data: {e}")

            ##Read Magnetic flux
            if not send_command_with_ok_check("$KE!"):
                print("Failed to send measurement command")

        # 行の終わりで次の行に移動する前にX軸をリセットする
        if y != y_num_points - 1:
            move_motor('X', -current_x)
            # 物体が位置に到達するのを待つ（調整が必要）
            time.sleep(1)
            current_x = 0.0
        
        ##Read Magnetic flux
        if not send_command_with_ok_check("$KE!"):
            print("Failed to send measurement command")

# ボタンがクリックされたときのイベントを処理する関数
def on_click(event):
    clear_output(wait=True)
    print("Starting the external program...")
    run_mesh_movement()
    serialInst.close()
    print("Program finished and serial port closed.")

# ボタンを作成
start_button = widgets.Button(description="Click here to start the program", layout=widgets.Layout(width='100%', height='100%'))
start_button.style.button_color = 'lightblue'

# ボタンクリックイベントを設定
start_button.on_click(on_click)

# Display Button
display(start_button)


Starting the external program...
No data received.
Sent command: $KE!
Response: #S000B00E9011500420011!#UFFFF18051E0B361E00E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#S000B00E9011500420011!#UFFFF18051E0B361F00E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#S000B00E9011500420011!#UFFFF18051E0B362000E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#S000A00E9011500420011!#UFFFF18051E0B362100E901000A!#S000A00E9011500420011!#S000A00E9011500420011!#S000A00E9011500420011!#UFFFF18051E0B362200E901000A!#S000B00E9011500420011!#S000A00E9011500420011!#UFFFF18051E0B362300E901000D!#S000B00E9011500420011!#S000C00E9011500420011!#UFFFF18051E0B362400E901000B!#S000B00E9011500420011!#UFFFF18051E0B362500E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#UFFFF18051E0B362600E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#UFFFF18051E0B362700E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#UFFFF18051E0B362800E901000B!#S000B00E9011500420011!#S000B00E9011500420011!#UF

### Voltage OFF

In [ ]:
# Power off all
out_put_off_all(rnd)

## Data Save

In [9]:
import csv
import os
import pandas as pd
from datetime import datetime

# Save folder
folder_path = "C:\\Users\\kary\\Documents\\9_data\\Electromagnet\\"  # Change path for yourself
file_path = os.path.join(folder_path, "data.csv")

# Initialize CSV file if not exists
def initialize_csv():
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "High_Precision_Timestamp", "X_Position", "Y_Position", "Magnetic_Flux", "Temperature", "Gauss_Value"])

# Add new columns to existing CSV
def add_columns_to_csv():
    df = pd.read_csv(file_path)
    if 'Magnetic_Flux' not in df.columns:
        df['Magnetic_Flux'] = ''
    if 'Temperature' not in df.columns:
        df['Temperature'] = ''
    if 'Gauss_Value' not in df.columns:
        df['Gauss_Value'] = ''
    df.to_csv(file_path, index=False)

# Save magnetic flux data to CSV
def save_magnetic_flux_to_csv(response_list):
    # Load existing CSV to DataFrame
    df = pd.read_csv(file_path)
    
    # Ensure the necessary columns exist
    if 'Magnetic_Flux' not in df.columns:
        df['Magnetic_Flux'] = ''
    if 'Temperature' not in df.columns:
        df['Temperature'] = ''
    if 'Gauss_Value' not in df.columns:
        df['Gauss_Value'] = ''
    
    # Add new data to the respective columns
    for i, response in enumerate(response_list):
        if i < len(df):
            temperature, gauss_value = parse_response(response)
            df.at[i, 'Magnetic_Flux'] = response
            df.at[i, 'Temperature'] = temperature
            df.at[i, 'Gauss_Value'] = gauss_value
        else:
            break
    
    # Save DataFrame back to CSV
    df.to_csv(file_path, index=False)

# Function to parse the response string
def parse_response(response):
    try:
        # Extract relevant parts of the response
        temperature_hex = response[18:21]  # 温度番号
        temperature_unit = response[22:23]  # 単位
        gauss_hex = response[24:27]  # ガウス値
        
        # Convert temperature from hex to decimal
        temperature = int(temperature_hex, 16)
        # Apply unit
        if temperature_unit == '1':
            temperature = temperature / 10
        
        # Convert gauss value from hex to decimal and adjust for sign and scale
        gauss_value = int(gauss_hex, 16)
        if gauss_value & 0x8000:
            gauss_value -= 0x10000
        gauss_value = gauss_value / 10
        
        return temperature, gauss_value
    except Exception as e:
        # Handle parsing error
        print(f"Error parsing response: {response}, error: {e}")
        return None, None

# 初期化と列の追加
initialize_csv()
add_columns_to_csv()

# データ取得コマンドを送信して応答を取得
data_response = send_receive_command("$A!")
if data_response:
    print(f"Data Response: {data_response.decode('latin-1')}")
else:
    print("No data received")

# 応答データをデータフレームに変換
if data_response:
    response_str = data_response.decode('latin-1')
    response_list = response_str.split('#')
    # 空のエントリを削除し、#Dで始まり!で終わるエントリのみを抽出
    response_list = [entry for entry in response_list if entry.startswith('D') and entry.endswith('!')]
    
    # データフレームに変換
    df = pd.DataFrame(response_list, columns=["response"])
    print(df)
    
    # 温度とガウス値を解析し、CSVに保存
    save_magnetic_flux_to_csv(response_list)
    print("Data saved to CSV")
else:
    print("No data to convert to DataFrame")

df


Sent command: $A!
Data Response: #S000B00EA011500420011!#UFFFF18051E0B373A00EA01000B!#S000B00EA011500420011!#S000B00EA011500420011!#S000D00EA011500420011!#UFFFF18051E0B373B00EA01000D!#S000D00EA011500420011!#S000D00EA011500420011!#S000B00EA011500420011!#UFFFF18051E0B380000EA01000D!#S000D00EA011500420011!#S000D00EA011500420011!#S000D00EA011500420011!#UFFFF18051E0B380100EA01000D!#S000D00EA011500420011!#S000D00EA011500420011!#S000B00EA011500420011!#UFFFF18051E0B380200EA01000B!#S000B00EA011500420011!#S000B00EA011500420011!#S000B00EA011500420011!#UFFFF18051E0B380300EA01000B!#S000B00EA011500420011!#S000B00EA011500420011!#S000D00EA011500420011!#UFFFF18051E0B380400EA01000D!#S000D00EA011500420011!#S000D00EA011500420011!#S000B00EA011500420011!#UFFFF18051E0B380500EA01000B!#S000B00EA011500420011!#S000B00EA011500420011!#S000B00EA011500420011!#UFFFF18051E0B380600EA01000D!#S000D00EA011500420011!#S000D00EA011500420011!#S000D00EA011500420011!#UFFFF18051E0B380700EA01000B!#S000B00EA011500420011!#S000B00EA

,response
0,D000018051E0B362900E901000B!
1,D000118051E0B362C00E901000D!
2,D000218051E0B362F00E901000B!
3,D000318051E0B363200E901000B!
4,D000418051E0B363500E901000B!
5,D000518051E0B363800EA01000B!
6,D000618051E0B370200EA01000B!
7,D000718051E0B370500EA01000B!
8,D000818051E0B370800EA01000B!
9,D000918051E0B370B00EA01000B!
